<a href="https://colab.research.google.com/github/IlyaKuprik/skolkovo-hack/blob/main/preprocessing_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Process

## загрузка


In [234]:
{"Самоотказ РЦ": 1, "Не отвечает": 2, "Отказ РЦ": 4, "Думает РЦ": 5, "Не пришел": 5, "Приглашен": 5, "Отклонен": 5, "Не пришёл": 5, 
  "Назначено собеседование в ресторан": 5, "Телефонное интервью": 5, "Пришел": 5, "Обзвон": 5, "ОТКАЗ СЦ": 5, "Отказ кандидата": 5, 
  "Отклик": 5, "Думает МП":5, "Расторгнул договор": 5, "Лендинг": 5, "Резерв": 5, "Отказ": 5}

{'Самоотказ РЦ': 1,
 'Не отвечает': 2,
 'Отказ РЦ': 4,
 'Думает РЦ': 5,
 'Не пришел': 5,
 'Приглашен': 5,
 'Отклонен': 5,
 'Не пришёл': 5,
 'Назначено собеседование в ресторан': 5,
 'Телефонное интервью': 5,
 'Пришел': 5,
 'Обзвон': 5,
 'ОТКАЗ СЦ': 5,
 'Отказ кандидата': 5,
 'Отклик': 5,
 'Думает МП': 5,
 'Расторгнул договор': 5,
 'Лендинг': 5,
 'Резерв': 5,
 'Отказ': 5}

In [237]:
data_candidates_work_places = pd.read_csv('/content/drive/MyDrive/hack/data/data_candidates_work_places.csv', sep=';',
                                names = ['CandidateId', 'Position','FromYear', 'FromMonth', 'ToYear', 'ToMonth'])

In [238]:
data_jobs = pd.read_csv('/content/drive/MyDrive/hack/data/data_jobs.csv', sep=';',
                                names = ['JobId', 'Status','Name', 'Region', 'Description'])

In [239]:
data_candidates = pd.read_csv('/content/drive/MyDrive/hack/data/data_candidates.csv', sep=';',
                                names = ['CandidateId', 'Position','Sex', 'Citizenship', 'Age',
                                'Salary', 'Langs', 'DriverLicense', 'Subway', 'Skills', 'Employment', 'Shedule',                                           
                                 'CandidateRegion', 'DateCreated', 'JobId', 'CandidateStatusId', 'Status'])

In [240]:
data_candidates_education = pd.read_csv('/content/drive/MyDrive/hack/data/data_candidates_education.csv', sep=';',
                                names = ['CandidateId', 'University', 'Faculty', 'GraduateYear'])

## группировка и создание последнего статуса

In [296]:
cols_val = {str(i): 'last' for i in data_candidates.columns}

In [297]:
grouped_with_last_status = data_candidates.sort_values(by = 'DateCreated').loc[:,data_candidates.columns]\
.groupby(by = ['CandidateId','JobId'], axis = 0, sort = False, as_index = False).agg( cols_val)

In [298]:
target_map = {"Самоотказ РЦ": 1, "Не отвечает": 0, "Отказ РЦ": 0, "Думает РЦ": 0.3, "Не пришел": 0.7, "Приглашен": 0.85, "Отклонен": 0.5, "Не пришёл": 0.8, 
  "Назначено собеседование в ресторан": 0.8, "Телефонное интервью": 1, "Пришел": 0.5, "Обзвон": 0.6, "ОТКАЗ СЦ": 0.3, "Отказ кандидата": 1, 
  "Отклик": 0.9, "Думает МП":0.4, "Расторгнул договор": 0.15, "Лендинг": 0.6, "Резерв": 0.5, "Отказ": 0.3}

In [299]:
targets = grouped_with_last_status['Status'].apply(lambda x: target_map[x] if x in target_map else np.NaN)
grouped_with_ids_target = grouped_with_last_status.loc[:, ['CandidateId', 'JobId',]]
grouped_with_ids_target['target'] = targets

In [300]:
grouped_table_candidates_features = grouped_with_last_status.drop(columns = ['Status', 'CandidateStatusId'])

In [301]:
grouped_table_candidates_features

,CandidateId,Position,Sex,Citizenship,Age,Salary,Langs,DriverLicense,Subway,Skills,Employment,Shedule,CandidateRegion,DateCreated,JobId
0,23669,Водитель,2,Гражданство,37,0,None,None,None,None,Full,Full,None,2014-08-29 00:00:00.0000000,369
1,23954,Водитель,0,None,0,0,None,None,None,None,Full,Full,None,2014-08-29 00:00:00.0000000,369
2,15143,Водитель,2,Россия,28,0,None,None,None,None,Full,Full,None,2014-03-04 13:04:20.0000000,292
3,15144,Водитель,2,Гражданство,39,0,None,None,None,None,Full,Full,None,2014-03-04 13:04:56.0000000,292
4,15127,Водитель,2,Гражданство,44,0,None,None,None,None,Full,Full,None,2014-03-04 13:04:46.0000000,292
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57353,24302656,None,2,None,22,0,None,None,None,None,Full,Full,None,2022-08-29 15:41:23.0000000,97611
57354,1674434,None,2,None,0,0,None,None,None,None,Full,Full,Москва,2022-08-29 16:03:38.0000000,97611
57355,24304149,None,2,None,32,0,None,None,None,None,Full,Full,Москва,2022-08-29 16:52:04.0000000,97611
57356,24304682,None,0,None,0,0,None,None,None,None,Full,Full,None,2022-08-29 17:18:06.0000000,333699


In [302]:
new_df = grouped_table_candidates_features\
    .merge(data_jobs, on='JobId')\
    .merge(data_candidates_education.drop_duplicates('CandidateId'), on='CandidateId', how='left')
new_df.index = [new_df.CandidateId, new_df.JobId]
df = new_df.drop(columns=['Region', 'Description', 'JobId', 'CandidateId'])
df.Sex = df.Sex\
    .where(~(df.Sex == 2), other='мужчина')\
    .where(~(df.Sex == 1), other='женщина')\
    .where(~(df.Sex == 0), other='неопределен')
df = df.fillna('NaN').astype(str).drop(columns = ["Status"]).add('. ').sum(axis=1).reset_index()
 
status_jobs = pd.DataFrame([[1, 'открытая'], [2, 'закрытая'], [3, 'приостановленная'], [4, 'отменена'], [5, 'черновик']],
                          columns = ['Status', 'name_status'])
new_data_jobs = data_jobs\
    .merge(status_jobs, on='Status')
new_data_jobs.index = new_data_jobs.JobId
job_user_tokens1 = new_data_jobs\
    .drop(columns=['Status', 'JobId', 'Description'])\
    .fillna('NaN')\
    .astype(str)\
    .add('. ')\
    .sum(axis=1)\
    .reset_index()
 
job_user_tokens2 = df
job_user_tokens_with_id = job_user_tokens2\
    .merge(job_user_tokens1, on='JobId')

In [303]:
merged_descriptions_targets_ids = job_user_tokens_with_id.merge(grouped_with_ids_target, on = ['JobId', 'CandidateId'], how = 'inner')
merged_descriptions_targets_ids.columns = ["CandidateId", "JobId", "Candidate_descr", "Jobs_descr", "target"]
merged_descriptions_targets_ids = merged_descriptions_targets_ids.fillna(1/2)

In [304]:
merged_descriptions_targets_ids

,CandidateId,JobId,Candidate_descr,Jobs_descr,target
0,23669,369,Водитель. мужчина. Гражданство. 37. 0. NaN. Na...,Водитель. Санкт-Петербург. закрытая.,0.5
1,23954,369,Водитель. неопределен. NaN. 0. 0. NaN. NaN. Na...,Водитель. Санкт-Петербург. закрытая.,0.5
2,23680,369,Водитель. мужчина. Россия. 27. 0. NaN. NaN. Na...,Водитель. Санкт-Петербург. закрытая.,0.5
3,23682,369,Водитель. мужчина. Россия. 25. 0. NaN. NaN. Na...,Водитель. Санкт-Петербург. закрытая.,0.5
4,23681,369,Водитель. мужчина. Россия. 35. 0. NaN. NaN. Na...,Водитель. Санкт-Петербург. закрытая.,0.5
...,...,...,...,...,...
57353,24191092,370097,Водитель ричтрака (погрузчика) на складе катег...,Водитель погрузчика / штабелера. Москва. откры...,0.5
57354,24191343,370097,Водитель погрузчика. мужчина. Россия. 48. 8000...,Водитель погрузчика / штабелера. Москва. откры...,0.5
57355,24207766,370097,Кладовщик-комплектовщик. мужчина. Россия. 35. ...,Водитель погрузчика / штабелера. Москва. откры...,1.0
57356,24207796,370097,"Водитель погрузчика, Штабелера. мужчина. Росси...",Водитель погрузчика / штабелера. Москва. откры...,1.0


In [293]:
merged_descriptions_targets_ids.to_csv('preprocess_data.csv')